# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages
from typing import Dict
import json
import csv
import re
import os
import glob

import cassandra
import pandas as pd
import numpy as np

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

In [6]:
# function to execute queries to cassandra
def execute_query(query: str, params: Dict = None):
    """Wrapper function around cassandra queries.
    
    If params Dict is passed the query is executed with the params datastructure.
    If any exception occurs, the exception is printed. The database response is returned.
    """
    try:
        if params:
            return session.execute(query, params)
        else:
            return session.execute(query)
    except Exception as e:
        print(e)

#### Create Keyspace

In [7]:
query = """
    CREATE KEYSPACE IF NOT EXISTS udacity_course 
    WITH REPLICATION = {'class': 'SimpleStrategy', 'replication_factor': 1}
"""

execute_query(query)

#### Set Keyspace

In [8]:
try:
    session.set_keyspace('udacity_course')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




# Question 1:
### Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

- The primary key of the table consists of the sessionId and the itemInSession as the two columns are needed for filtering, also the combination of the two columns is unique.
- The sessionId is used as partition key and itemInSession as clustering key.
- artist, song and length were added to the table as the values are asked for by the business question.


In [9]:
# Queries to answer question1, named place-holders were used in the insert query to prevent errors due to false order of arguments.

create_query_1 = """
    CREATE TABLE IF NOT EXISTS sessions (
        sessionId int, 
        itemInSession int, 
        artist text,
        song text,
        length float,
        PRIMARY KEY (sessionId, itemInSession)
    )
"""
insert_query_1 = """
    INSERT INTO sessions (sessionId, itemInSession, artist, song, length)
    VALUES (%(session_id)s, %(item_in_session)s, %(artist)s, %(song)s, %(length)s)
"""
select_query_1 = """
    SELECT artist, song, length
    FROM sessions
    WHERE sessionId = 338
    and itemInSession = 4
"""
drop_query_1 = """DROP TABLE sessions"""


In [10]:
# Create table sessions to answer question1:
result = execute_query(create_query_1)

In [11]:
# Fill table sessions. csv.DictReader and a dict params-datastructure were used to prevent errors due to false indices or false order of arguments.

with open('event_datafile_new.csv', encoding = 'utf8') as f:
    csvreader = csv.DictReader(f)
    next(csvreader) # skip header
    for line in csvreader:
        params = {
            "session_id": int(line['sessionId']),
            "item_in_session": int(line['itemInSession']),
            "artist": line['artist'],
            "song": line['song'],
            "length": float(line['length'])
        }
        
        execute_query(insert_query_1, params)

In [12]:
# Query for table sessions
for row in execute_query(select_query_1):
    print(row)

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)


# Question 2:
### Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

- The primary key of the table consists of the userId, sessionId and the itemInSession.
- userId and sessionId are needed for filtering and form the compound partition key.
- itemInSession is needed as a clustering key as the retrieved songs should be sorted by itemInSession.
- artist, song, firstName and lastName were added to the table as the values are asked for by the business question.


In [13]:
# Queries to answer question2, named place-holders were used in the insert query to prevent errors due to false order of arguments.

create_query_2 = """
    CREATE TABLE IF NOT EXISTS user_sessions (
        userId int, 
        sessionId int, 
        itemInSession int,
        artist text,
        song text,
        firstName text,
        lastName text,
        PRIMARY KEY ((userId, sessionId), itemInSession)
    )
"""
insert_query_2 = """
    INSERT INTO user_sessions (userId, sessionId, itemInSession, artist, song, firstName, lastName)
    VALUES (%(user_id)s, %(session_id)s, %(item_in_session)s, %(artist)s, %(song)s, %(first_name)s, %(last_name)s)
"""
select_query_2 = """
    SELECT artist, song, firstName, lastName
    FROM user_sessions
    WHERE userId = 10
    and sessionId = 182
"""
drop_query_2 = """DROP TABLE user_sessions"""

In [14]:
# Create table user_sessions to answer question2:
result = execute_query(create_query_2)

In [15]:
# Fill table user_sessions. csv.DictReader and a dict params-datastructure were used to prevent errors due to false indices or false order of arguments.

with open('event_datafile_new.csv', encoding = 'utf8') as f:
    csvreader = csv.DictReader(f)
    next(csvreader) # skip header
    for line in csvreader:
        params = {
            "user_id": int(line['userId']),
            "session_id": int(line['sessionId']),
            "item_in_session": int(line['itemInSession']),
            "artist": line['artist'],
            "song": line['song'],
            "first_name": line['firstName'],
            "last_name": line['lastName'],
        }
        
        execute_query(insert_query_2, params)

In [16]:
# Query for table user_sessions
for row in execute_query(select_query_2):
    print(row)

Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


# Question 3:
### Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

- The partition key of the table consists of the song as it is needed for filtering.
- In addition the userId is used as clustering key to achieve a unique primary key for each row.
- firstName and lastName were added to the table as the values are asked for by the business question.

In [17]:
# Queries to answer question3, named place-holders were used in the insert query to prevent errors due to false order of arguments.

create_query_3 = """
    CREATE TABLE IF NOT EXISTS song_listeners (
        song text,
        userId int,
        firstName text,
        lastName text,
        PRIMARY KEY (song, userId)
    )
"""
insert_query_3 = """
    INSERT INTO song_listeners (song, userId, firstName, lastName)
    VALUES (%(song)s, %(user_id)s, %(first_name)s, %(last_name)s)
"""
select_query_3 = """
    SELECT firstName, lastName
    FROM song_listeners
    WHERE song = 'All Hands Against His Own'
"""
drop_query_3 = """DROP TABLE song_listeners"""

In [18]:
# Create table user_sessions to answer question3:
result = execute_query(create_query_3)

In [19]:
# Fill table user_sessions. csv.DictReader and a dict params-datastructure were used to prevent errors due to false indices or false order of arguments.

with open('event_datafile_new.csv', encoding = 'utf8') as f:
    csvreader = csv.DictReader(f)
    next(csvreader) # skip header
    for line in csvreader:
        params = {
            "song": line['song'],
            "user_id": int(line['userId']),
            "first_name": line['firstName'],
            "last_name": line['lastName'],
        }
        
        execute_query(insert_query_3, params)

In [20]:
# Query for table user_sessions
for row in execute_query(select_query_3):
    print(row)

Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Tegan', lastname='Levine')
Row(firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

In [21]:
# Drops all the tables.
result1 = execute_query(drop_query_1)
result2 = execute_query(drop_query_2)
result3 = execute_query(drop_query_3)

### Close the session and cluster connection¶

In [22]:
session.shutdown()
cluster.shutdown()